## Part 1 

We need to scan the grid and find which rolls (`@`) have fewer than 4 rolls of paper within the 8 square neighborhood.

In [1]:
from pathlib import Path
from PIL import Image, ImageDraw

import helper as h

logger = h.setup_logger("AoC_2025: Day 4")

In [2]:
import numpy as np
import re
def read_input(src: str | Path):
    if Path(src).is_file():
        with open(src, "r") as f:
            data = f.read().strip().splitlines()
    else:
        data = src.strip().splitlines()

    grid_mat = np.zeros((len(data), len(data[0])), dtype=np.uint8)
    for i, line in enumerate(data):
        for match in re.finditer("@", line):
            j = match.start()
            grid_mat[i][j] = 1
    
    return grid_mat

test_grid = """
@.@.
.@..
..@.
"""
test_mat = np.array([[1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0]])

h.extended_assert(test_grid, read_input(test_grid), test_mat)

In [3]:
def fewer_than_four(grid_mat: np.ndarray):
    roll_count=0
    max_i, max_j = grid_mat.shape
    for i in range(max_i):
        for j in range(max_j):
            if grid_mat[i][j] == 0:
                continue

            # get all is and js 

            min_x = max(0, i - 1)
            max_x = min(i + 2, max_i)
            min_y = max(0, j - 1)
            max_y = min(j + 2, max_j)

            slice = grid_mat[min_x:max_x, min_y:max_y]

            if np.sum(slice) - 1 < 4:
                roll_count += 1
    
    return roll_count

def part1(src: str | Path):
    gm = read_input(src)
    return fewer_than_four(gm)

h.extended_assert("toy", part1("./inputs/day_04_toy.txt"), 13)


In [4]:
part1("inputs/day_04.txt")

1564

## Part 2

In this part, we gradually remove the rolls. Once we access all available, we remove them. Then we scan for more accessible, remove them etc. Until we can't remove any.

In [5]:
def remove_rolls(grid_mat: np.ndarray):
    roll_count = 0
    max_i, max_j = grid_mat.shape
    update_mat = grid_mat.copy()
    while True:
        change = 0
        for i in range(max_i):
            for j in range(max_j):
                if grid_mat[i][j] == 0:
                    continue

                min_x = max(0, i - 1)
                max_x = min(i + 2, max_i)
                min_y = max(0, j - 1)
                max_y = min(j + 2, max_j)

                slice = grid_mat[min_x:max_x, min_y:max_y]

                if np.sum(slice) - 1 < 4:
                    change += 1
                    update_mat[i][j] = 0

        grid_mat = update_mat.copy()

        roll_count += change
        if change == 0 or np.sum(update_mat) == 0:
            break

    return roll_count


def part2(src: str | Path):
    gm = read_input(src)
    return remove_rolls(gm)


h.extended_assert("toy", part2("inputs/day_04_toy.txt"), 43)

In [6]:
def part2(src: str | Path):
    gm = read_input(src)
    return remove_rolls(gm)

h.extended_assert("toy", part2("inputs/day_04_toy.txt"), 43)

In [7]:
part2("inputs/day_04.txt")

9401

In [8]:
def grid_to_image(grid_mat: np.ndarray, cell_px: int = 20) -> Image.Image:
    """
    Convert 0/1 grid to a scaled PIL image.
    cell_px = how many pixels each grid cell should be on a side.
    """
    mat = grid_mat.astype(np.uint8)

    # white background, black rolls (1 → 0, 0 → 255)
    base = 255 - mat * 255  # still shape (h, w), dtype uint8
    img = Image.fromarray(base, mode="L")

    if cell_px > 1:
        h, w = mat.shape
        img = img.resize((w * cell_px, h * cell_px), resample=Image.NEAREST)

    return img


In [9]:
def remove_rolls_gif(grid_mat: np.ndarray, gif_path: str | Path, cell_px: int = 20, duration: int = 250):
    frames = []
    grid_mat = grid_mat.copy()
    update_mat = grid_mat.copy()
    max_i, max_j = grid_mat.shape

    # initial frame
    frames.append(grid_to_image(grid_mat, cell_px))

    while True:
        change = 0
        for i in range(max_i):
            for j in range(max_j):
                if grid_mat[i][j] == 0:
                    continue

                min_x = max(0, i - 1)
                max_x = min(i + 2, max_i)
                min_y = max(0, j - 1)
                max_y = min(j + 2, max_j)

                slice_ = grid_mat[min_x:max_x, min_y:max_y]

                if np.sum(slice_) - 1 < 4:
                    change += 1
                    update_mat[i][j] = 0

        frames.append(grid_to_image(update_mat, cell_px))

        grid_mat = update_mat.copy()

        if change == 0 or np.sum(update_mat) == 0:
            break

    gif_path = Path(gif_path)
    frames[0].save(
        gif_path,
        save_all=True,
        append_images=frames[1:],
        duration=duration,
        loop=0,
    )


In [10]:
gm = read_input("inputs/day_04_toy.txt")
remove_rolls_gif(gm, "day4_toy.gif", cell_px=20)

![](./day4_toy.gif)

In [11]:
gm = read_input("inputs/day_04.txt")
remove_rolls_gif(gm, "day4.gif", cell_px=20)

![](./day4.gif)